# 東森新聞雲爬蟲練習
## 練習從東森新聞雲網站中，爬取新聞摘要及詳細內容。
## 學習利用Selenium模擬人為操作，更新動態網頁後爬取新聞內容。

In [1]:
import requests
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

### 以財經新聞為例，先連結到財經新聞網頁，複製其URL。

In [2]:
ETtoday_url = "https://www.ettoday.net/news/focus/%E8%B2%A1%E7%B6%93/"  #財經新聞

### 用Selenium打開一個瀏覽器，連結到網站。然後模擬下拉網頁的動作，讓網頁完成更新，再處理後續動作。

In [3]:
browser = webdriver.Chrome(executable_path='../chromedriver')
browser.get(ETtoday_url)  # 打開瀏覽器並連到東森新聞雲網頁

SCROLL_PAUSE_TIME = 1

#
# 以下是用Selenium模擬下拉網頁動作，讓網頁更新
#
last_height = browser.execute_script("return document.body.scrollHeight")
while True:
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    print('網頁更新中...')
    time.sleep(SCROLL_PAUSE_TIME)

    new_height = browser.execute_script("return document.body.scrollHeight")
    # 判斷是否到底
    if new_height == last_height:
        print('到達頁面底端')
        break
    last_height = new_height

# 參考：
# https://stackoverflow.com/questions/48850974/selenium-scroll-to-end-of-page-indynamically-loading-webpage/48851166

網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
到達頁面底端


### 到這裡網頁已經更新完畢，所有的新聞都已經出現在網頁上。接下來做爬取收集新聞的動作。

In [4]:
#
# 這裡先建立一個函數，其功能是連到外部連結，並爬取新聞內容。
#
def getNewsDetailContent(link_url):
    resp = requests.get(link_url)
    resp.encoding = 'utf-8'
    #print(resp.text)

    soup = BeautifulSoup(resp.text, 'lxml')
    news_content = soup.find("div", attrs={'class':'story'}).find_all("p")
    for p in news_content:
        """
        .string屬性說明：
        (1) 若當前tag節點底下沒有其他tag子節點，會直接抓取內容(返回"NavigableString")
        (2) 若當前tag節點底下只有唯一的一個tag子節點，也會直接抓取tag子節點的內容(返回"NavigableString")
        (3) 但若當前tag節點底下還有很多個tag子節點，.string就無法判斷，(返回"None")
        """
        if ((p.string) is not None):
            print(p.string)

### 解析HTML並萃取新聞摘要，若有外部的連結，再連到外部連結並把詳細新聞內容抓取下來。

In [5]:
# 爬取網頁內容，解析後萃取新聞摘要
html = browser.page_source
soup = BeautifulSoup(html, "lxml")
all_news = soup.find("div", attrs={'class':'block block_1 infinite_scroll'})

news_block = all_news.find_all('div', attrs={'class':'piece clearfix'})

for i, news_item in enumerate(news_block):
    print("----------------------------------------------------------------------")
    news_body = news_item.find('h3')
    print("\n[%d] %s\n" % (i, news_body.a.string))
    
    # 
    # 連到外部連結，擷取詳細新聞內容
    #
    externalLink = "https://www.ettoday.net" + news_body.a["href"]
    getNewsDetailContent(externalLink)

browser.close()

----------------------------------------------------------------------

[0] 台大歷史博士鑽研投資　只靠ETF十年養出千萬財

圖文／鏡週刊
身為德明財經科技大學通識中心助理教授的張遠，在網路上以研究分析哆啦A夢聞名，更被網友封為「哆啦王」，身為台大歷史學系博士的他，不光看漫畫，也從中做學問，私底下，他還是個熱愛研究投資理財的高手，是被動式投資的信仰者，投資ETF超過10年，如今已累積逾千萬元投資部位，去年整體獲利12.5%。
張遠表示，當年大學聯考，因為自己從小對歷史有興趣，所以第一志願是台大歷史系，商管學院則被他排在最後，「還好我爸爸是台大中文系教授，否則一般家庭應該很難接受小孩明明可以選念法律系，卻跑去讀歷史系。」他笑說，自己也沒料到繞了一大圈，現在會對年輕時最無感的投資熱衷研究。
從小愛看漫畫的張遠透露，自己有一部分的投資體悟，就是從《哆啦A夢》漫畫學到的。《哆啦A夢》漫畫裡曾經出現過一個叫「增多的花盆」道具，不管什麼東西，只要種進花盆裡，就會增多，哆啦A夢和大雄把蛋糕種進去，澆水之後，很快就發芽、結果，他們把果實打開就出現一模一樣的蛋糕，把1個蛋糕增多變成10個蛋糕。
同學胖虎知道後，要求大雄借他道具，他想成為有錢的大富翁，於是把錢種進花盆裡，翹腳躺著準備等錢增多，可是最後結出來的果實卻很小，打開只出現像郵票大小的錢鈔，根本不能用，原來，胖虎沒有幫花盆澆水，果實就變成營養不良的小果。
「道具跟投資商品的概念其實是相同的，再好的道具，如果沒有了解它並妥善應用，結果也不會如願。」這是哆啦王從漫畫中得來的投資體悟。
----------------------------------------------------------------------

[1] 中火爭議再起　經長王美花向中市府喊話：不要為難基層員工

▲中火爭議再起，經濟部長王美花今（25）日表示，台電啟動二號機是依法辦理。（圖／記者林敬旻攝）
記者林淑慧／台北報導
中火爭議再起，針對台中市政府環保局稱，將對台中火力發電廠違法開罰一事，經濟部長王美花今（25）日表示，台電啟動二號機是依法辦理，希望中市府不要為難基層員工，強調為炎炎夏日做穩定供電之準備，是完全合法的行為。
近日天氣炎熱、疫情後產業活動也持續復甦，昨（24

記者林淑慧／台北報導
台中市政府將對台中電廠2號機點火啟動開罰，台電回應表示，機組處於許可證申請展延狀態、依原條件操作「合法」，對於市府裁罰處分，台電將依法提出救濟。
▲台電擴大燃煤電廠降載，台中電廠降載量為去年5.4倍。圖為台中電廠。（圖／台電提供）
台電說明，有關合法性議題，台電願不厭其煩再次強調，台中電廠兩部機組雖曾遭市府裁罰廢證，但因該廢證裁罰違反空污法規定，環保署已依法撤銷該廢證裁罰，恢復原許可證效力，就算市府已提起行政救濟，撤銷裁罰的效力不因此中止，而市府環保局所提訴願亦經行政院訴願委員會做出不受理之決定。
依空汙法規定，機組回到許可證申請展延狀態，即可依原許可證內容及許可條件進行操作。市府提出許多令人費解的法律解釋，刻意曲解法律，甚至不經法律程序，直接「斷言」中央主管機關撤銷裁罰處分無效，此舉反而已對國家法治造成重大傷害，更重創人民對於政府行政處分效力的信任。
台電指出，台電從未拒絕遵守法律，更是比誰都關心空品議題。台電只是希望地方政府能理解，穩定供電與改善空品從來不是二選一，台電不會逼民眾選擇穩定供電而放棄空氣品質，但也希望台中市政府不要逼人民放棄用電權益。人民有呼吸新鮮空氣的權利，人民也有使用電力追求生活品質的權利。台電堅持，這兩項權利都應該尊重。
▲經濟部政務次長曾文生（圖中）日前表示，中火未來全廠10部機組不再「全員到齊」，而是安排「先發陣容」，派出狀態最好的機組上線發電，預計台中電廠減排幅度將達六至七成。（圖／記者林淑慧攝）
台電強調，台中電廠2號機空品改善完成，排放符合台中市政府制定的加嚴標準。台中市政府如果真的有心改善空污，應該將眼光放到其他污染源上頭，切勿因爲過度將注意力放在台中電廠上，而錯失了處理其他空污來源的契機，對改善空品反而毫無助益。
台電表示，一個確實能兼顧穩供與環保的發電機組，為了滿足人民用電需求進行合理調度，竟遭到台中市政府視若仇敵，多次威嚇對待，實在情何以堪。台電仍呼籲市府放下成見，共尋良方，不要再將穩供與環保操作成零和的政治遊戲。
►過期票券別浪費！簡單3步驟　教你搶救妙招
----------------------------------------------------------------------

[13] 威力彩連槓38期端午頭獎衝10億　今年最大獎靠「意境牌」終結

記者紀佳妘／台北報導
威力彩連

▲美股大跌。（圖／路透社）
記者張靖榕／綜合外電報導
美股周三（24日）大跌，美國新冠肺炎疫情持續高漲，甚至在第一波未平之前就引起第二波來襲的憂慮，再加上美國宣布對法、德、西、英等歐洲進口產品加徵關稅，產品價值31億美元，又一次撩起美歐關係緊張，美股三大指及歐洲三大股，全大跌收場。
道瓊工業指數終場暴跌710.16點，或2.72%，以25445.94點作收；標普500指數下跌80.96點，或2.59%，收在3050.33點；那斯達克指數則是下跌222.2點，或3.11％，以9909.17點作收。
歐股方面，英國FT100下跌196.43點，或3.11％，以6123.69點作收；NQ德國指數下跌42.86點，或3.87％，以1064.13點作收；法國CAC40指數則是下跌146.32點，或2.92％，收在4871.36點。
美國宣布對歐盟、英國加徵關稅，讓原本就因為新冠肺炎疫情難平的市場更加憂慮，不安因素增加後，直接反應在股市上。而截至台灣時間25日上午4點30分，美國累計已達236萬9806例、12萬1846死；總統川普先前在土爾沙（Tulsa）的造勢大會上曾表示，美國每篩檢一個人，就會多一個病例，「所以我叫他們慢一點」！
----------------------------------------------------------------------

[19] 千億豪門爭產鬥！天上掉下來的千億寶藏　柯市府急脫手原因曝光

圖文／鏡週刊
目前唯一合法可以管理頴川建忠千億財產的台北市政府社會局，2018年在法官裁定「輔助宣告後」，接手這件史無前例的千億元財產監管案，過程戰戰兢兢，深怕一個不小心，也會成為頴川家族爭產下的被告。
 
由於頴川的資產龐大，加上遍布海內外，子女間又有不同意見，社會局不想捲入雙方的爭奪戰，已向台北地院表達希望另請高明，由法院選定律師或會計師等專業人員管理，此案目前由台北地院家事庭審理中。
頴川建忠的5個子女與老臣，為了爭產已分裂成2派，次子頴川欽和陣營反對由哥哥頴川浩和派的任何人擔任管理父親財產的輔助人，基於雙方無法共同推舉中立、無利害關係的第三人擔任頴川建忠的輔助人，法院決定依「當事人最大利益」原則，2018年2月裁定由台北市社會局當輔助人，協助管理頴川建忠的財產，但仍無法平息二派糾紛。
頴川建忠的子女2016年9月帶著父親到台大

籌碼達人陳威良表示，通過解盲後接著還有藥證的問題，還有上市銷售後取得多少市佔跟產品定價等。陳威良認為買合一要把它當成買選擇權、樂透的概念，「這筆錢你可以輸得起，那你去買。但一支飆股也不太容易在分盤期間就結束行情。」
下一檔生技黑馬是誰？
阮慕驊觀察，新藥題材很常出現宣布解盲成功後股價不升反降的情況，因此他認為，在生技股操作方面，應著重於那些還沒解盲的公司。「當然，這就有點押寶了，押它解盲會成功，這就有點賭博性質，因為到底解盲會不會成功也不知道。」
陳威良則認為，現在最實際就是經濟解封、航線重啟之後的大量篩檢需求。他預估，全球的檢測需求會從今年的15億人次暴增到明年的55億人次，市場規模上看585億美元。「重點是，國內切入這塊市場的公司其實股本都非常迷你，這些小公司進到一個大的藍海市場，只要稍微抓到一部分市場，它的EPS就會往上大幅度的跳升了。」
▲陳威良指出，選擇生技股投資標的可觀察其生產的新冠肺炎檢測產品所對應的檢測方式。（圖／截自《雲端最有錢》）
陳威良舉例，像普生（4117）已於4月拿到歐盟許可、5月拿到印度許可，5月底也已送美國申請，「像興櫃的金萬林或最近剛上市不久的瑞磁（ABC-KY），拿到美國許可後，股價都有滿明顯的強漲跡象。」他指出，投資生技股有3大觀察重點，包括其肺炎檢測方式、上市是否已拿到許可或相關進度，另外還有銷售地點，像印度等新興國家的市場就較歐美更大。
----------------------------------------------------------------------

[28] 鴻海除息交易日訂7/23　現金股利4.2元預計8/21發放

▲鴻海土城總部。（圖／記者姚惠茹攝）
記者姚惠茹／台北報導
鴻海（2317）今（24）公告，今年每股配發現金股利4.2元，訂定除息交易日為7月23日，並預計將股利發放日為8月21日，今年總計將發出582.25億元的現金股利。
鴻海公告，除息基準日訂於7月31日，普通股最後過戶日為7月26日，停止過戶期間為7月27日至31日，海外可轉換公司債停止轉換期間為7月6日至7月31日。
鴻海今年每股配發現金股利4.2元，若以今日收盤價83.5元計算，現金殖利率達5%，而鴻海創辦人郭台銘目前持有鴻海股票134.17萬張，預計將能進帳56.35億元的現金股利，至於鴻海董事長劉揚偉則持股656張，

文／陳文暉
行政院6月19日宣布，經濟部長由法律智財專業的經濟部政務次長王美花接掌，20日正式交接，蔡政府新任期的內閣改組自此告一段落。據了解，早在520之前，蔡政府就思考罷韓成功後，陳其邁返回高雄選舉，空出副閣揆職缺所牽動的人事布局。
小英屬意黃志芳？　蘇貞昌堅持內部擢升
「對各派系最無害、沒有威脅」，黨政人士直言，這是原經長沈榮津在綠營派系林立中，可以勝出的主因，且沈榮津對長官交付任務使命必達，口罩國家隊的成功，讓府院二巨頭對這位「歐吉桑」更加信任，成為二人最大公約數。但找誰掌舵下一任經長，才是府院最傷腦筋的事。
因為歷經工研院董事長李世光出任蔡政府第一任經長時，短短一年三個月，就為限電扛責下台陣亡，府院高層及核心政治幕僚都嚇到了。為避免前車之鑑，經長一職，蘇揆堅持內升不考慮派系政治酬庸，小英則要求人選以「穩健」為主，據悉府院對經長人選曾有番角力與拉鋸。
知情人士透露，找人過程經過不只一輪討論，蔡、蘇二巨頭各有所屬，傳出小英屬意貿協董事長黃志芳、蘇揆屬意內升政次王美花；但黃本人並無意願，就連商業發展研究院董事長許添財，油、電、糖等國營事業董事長，都一一盤點過，也找不出令層峰放心的合適人選。
黨政知情人士透露，府方比較擔心王美花的政治判斷能力，因此猶豫。儘管有老公顧立雄作為堅強後盾，但政治判斷必須立即應變、危機處理，面對立院咄咄逼人需有臨場反應，必須要靠個人政治智慧及機智反應，但王美花這一部分略顯不足，令黨內人士憂心。
不過，有政治判斷的人，未必管得住經濟部這個龐大機器。經濟部長本來就是個「政治風險極高」的官位，業務包山包海，錯綜複雜，歷年因油電水等民生事件下台的經長或國營事業董事長不在少數，到處都是「地雷」，而且產業、能源相關主管機關眾多，事業單位也各有山頭，不但最起碼不能出包，更是政府的重要工具，顯見要掌舵經濟部，專業及政治門檻都不低。核心幕僚及高層研商時，第一個念頭就鎖定「從經濟部系統找人」，且掃描人才庫的過程，不難窺見「資深」的關鍵字。
延伸閱讀：
蘇貞昌大動作強勢主導內閣改組 蔡英文力保「這三名」愛將維持恐怖平衡
一手打造台灣口罩國家隊 謝金河讚他：難得的好官
----------------------------------------------------------------------

[40] 證交所股東會通過配發股利2.7元　創10年

▲張琮閔和時下年輕人不一樣，他努力賺錢不是為了物質慾，而是為了一家團圓。
圖、文／永慶房屋提供
「你必須很努力，才能看起來不費力。」24歲的張琮閔，畢業於逢甲大學都市計畫與空間資訊學系，他的人生目標和時下年輕人很不一樣，認真工作不是為了換最新的iPhone、或是買新上市的PS5，而是拚5年內在台北市買房，許多人好奇為何年紀輕輕就有買房目標？張琮閔說：「因為我想要一家人團圓。」
因為家人工作緣故，張琮閔在國小2年級時舉家搬到廣東，直到高中畢業後才自行回台就讀大學，和家人分隔兩地。每當看到家人兩地辛苦奔波，只為了一家溫飽，他在心中默默許下心願，一定要認真工作報答家人的付出，並且努力能夠早日一家團聚。
張琮閔小時候跟在製鞋廠工作的父親到處趴趴走，每天和不同單位的長輩們交際互動，養成外向、活潑的性格。大學時利用暑假到食品銷售公司實習，從行政庶務、商品介紹到陪同業務拜訪客戶，開啟他對業務工作的興趣。大學畢業時，他認真思考一個問題：「業務工作百百種，到底哪一種最適合我？」
▲張琮閔回台灣就讀大學，和父母分隔兩地，他心中嚮往未來能夠一家團聚。
社會新鮮人免擔心！永慶房屋保障前9個月每月5萬元
身上背著50萬的學貸，對剛畢業的社會新鮮人來說壓力不小。張琮閔在找工作時相當苦惱，大學畢業生平均起薪多只有2萬6到2萬8，再扣掉生活費，真的不知道要何年何月才能還清學貸，更別提實現他的買房夢。一次和大學好友聚餐，好友分享他在永慶房屋的工作及公司福利，讓張琮閔聽得心癢癢：「永慶房屋保障業務新人前9個月每月5萬元，比起其他行業待遇高出太多了！」他仔細盤算，若再加上每年永慶房屋留才的「幸福成家基金」，年薪破百萬根本不是問題。
原本還在猶豫到底要賣保險、賣車還是賣房子，比較後他知道加入永慶房屋能讓他離夢想更近一步。當父母聽到張琮閔的選擇時，也支持兒子趁著年輕衝刺事業，相信在永慶房屋的品牌加持下，張琮閔絕對能走出自己的康莊大道。
▲媽媽(圖中)是張琮閔(圖右)心中最溫柔的力量，支持他全心投入房仲工作。
師徒制一對一指導！彈性工作8小時兼顧工作與生活
「如果經紀人被客戶問倒，場面一定很尷尬！」張琮閔笑說，好在永慶房屋提供360個小時教育訓練，幫他建立完整的房地產專業知識，還有「師徒制」透過資深學長姐一對一指導，從實戰中累積服務客戶的經驗與技巧，讓他在短短半年內就有成交，入職11個月來，平均月薪6萬元以上

▲6月18日，央行召開今年第二季度理監事會議，最終決定「維持政策利率不變」。（圖／記者黃克翔攝）
●林建甫／中信金融管理學院講座教授、台大經濟系名譽教授
6月18日我國中央銀行的理監事會議，並沒有降息，有點跌破市場眼鏡。重貼現率維持1.125%，仍是歷史新低。倒是央行倒是下修今年經濟成長預估為1.52%，低於前次央行理監事會議預估的1.92%，也低於主計總處五月底預測的1.67%。這點符合市場預期。
市場上本來有不少預期央行將降息半碼，甚至一碼的可能性。原因是台灣第二季經濟成長率很可能較第一季下滑，產出負缺口持續擴大；整體和核心 CPI 年增率仍在下滑中，構成繼續降息的條件。而前一星期觀察IRS（利率交換）走勢，長天期利率報價低於短天期、呈「負斜率」，某種程度反映市場對央行降息的預期。
期望降息也反映國際資金市場的連動性。美國聯準會（Fed）率先降到幾乎零利率後，為避免被套利，各國央行不能不跟，陸續的降息。我們央行雖然三月的理監事會議也打破連14凍的降了一碼，但在國際潮流下，仍嫌不足，恐怕仍有套利空間，因此需要再度降息來弭平。
另外，今年以來，新台幣已出現強勢格局，根據新台幣名目有效匯率指數，第二季新台幣名目有效匯率指數仍呈現上升趨勢，走勢強壓韓元，不利出口廠商海外報價競爭力；而利差因素，一再推升已經處在波段新高的新台幣匯率，進而可能讓熱錢湧入台灣；加上強勢匯率更進一步亦構成壓抑國內物價壓力。因此降息有助於避免通縮，甚至創造溫和通膨，促進經濟繁榮。
▲台灣一度接近被美國財政部列入匯率操縱國名單。（圖／達志影像／美聯社）
但由於目前台灣利率也是非常低的狀態，降息空間所剩無幾的情況下，央行子彈省著點用，未來如果疫情再度反撲或世界經濟有超乎意料的回檔時，傳統貨幣政策仍保有作用的空間。而且從近期定存單、公債利率探底可看出，市場資金水位充沛，並無再度降息的必要性跟急迫性。
▲降息對經濟振興效果有限，且可能陷入「流動性陷阱」。（圖／記者黃克翔攝）
過去主張降息的論點是提振經濟，讓市場重回繁榮的軌道。理論上是利率下降，影響家計單位與廠商之消費及投資決策，達到刺激總合需求的目標，進而使經濟轉好。這是貨幣學的主要訴求及貨幣政策傳遞管道。但現在降息對經濟提振有限，且副作用倒是不小。
為什麼經濟提振有限？凱因斯很早就提出「流動性陷阱」。利率太低，人們寧願持有現金，投機性貨幣需求的利率彈性

▲金管會主委黃天牧。（圖／記者林敬旻攝）
記者紀佳妘／台北報導
在疫情趨緩下，金管會解除限空令，國安基金是否退場也受到市場矚目，主委黃天牧今（24）日表示，政經情勢不斷變動，股市「變是常態」，黑天鵝、灰犀牛將會持續發生，但他仍對股市相當有信心。另為解決國內「殭屍股」橫行問題，預計明（2021）年6月底提出「造市者流動性制度」。
金管會主委黃天牧今天接受媒體聯訪時表示，隨著政治、經濟、金融環境不斷變動，股市波動是正常的，「變是常態」，黑天鵝、灰犀牛仍會發生，但重點在於金融機構是否有足夠韌性因應，由於資本市場走了多年，投資人也相對成熟，1700多家上市櫃公司的公司治理、基本面、殖利率等績效有目共睹，「我對股市相當有信心」。
根據金管會統計，台灣與全球主要市場股價指數相比，台股今年以來跌幅達3.21%，相較於香港跌幅11.64%、新加坡跌幅18.24%、日本跌幅4.68%、美國道瓊跌幅8.35%，表現相對穩定，僅略低於上海跌幅2.61%、韓國3.02%，而深圳成指則上揚13.07%。
黃天牧指出，既然有信心，即便有短暫波動，但仍要回歸基本面，儘管現階段已解除限空令，但市場若出現非理性或有較大變化時，主管機關站在穩定市場的前提下，也會做出必要的作為。
證期局長張振山也認為，「危機就是轉機」，尤其在全球市場變動時，國內疫情相對穩定，除了基本面表現不錯外，觀察近3個月的日均交易量表現，4月1600億元、5月1700億元、6月1900億元，到昨天更來到2000億元，顯示市場穩定時，投資人有信心，也不必過度擔憂國際衝擊，足以抵抗外來的黑天鵝、灰犀牛。
張振山表示，針對雞蛋水餃股已有訂定下市機制，但如何讓體質不錯，但流動性少的「殭屍股」活絡起來，目前研擬推動造市者流動性制度，並祭出足夠誘因，期盼券商扮演台股造市者的角色，現階段已有業者意願加入，由於證交所仍在規劃時程，預計明（2021）年6月底出爐。
另在疫情衝擊下，為減緩企業申請IPO掛牌影響，黃天牧也提及，今年設定IPO家數共有52家，但上半年IPO掛牌的企業只有9家，因此預計7月祭出寬容措施，期許透過鬆綁IPO掛牌條件，讓原先因疫情無法順利上市櫃公司盡量降低衝擊，目前看起來到年底，「不敢說百分百52家都可以」，但會盡量努力達標。
--------------------------------------------------

記者蔡紹堅／綜合報導
香港特首林鄭月娥23日晚間在中國發展高層論壇網絡簡報會上致詞，她表示，香港的聯繫匯率未來將持續維持穩定，不需要美國批准去實行此制度，亦無計劃作出改變，「我們有能力捍衛任何對聯繫匯率的惡意攻擊。」
林鄭月娥指出，在沒有相關的法律制度和執行機制下，香港已成為國家安全的缺口，加上從去年6月，在外部勢力煽動下，暴力不斷升級，香港飽受創傷，情況變得不能接受，多宗涉及私藏彈藥和槍械的案件，構成恐怖主義風險，嚴重影響公共安全，也有鼓吹「港獨」、「自決」的組織煽動示威者，特別是青少年，公然污衊國旗、損毁國徽和衝擊中央駐港機構，「這些行為詆毁『一國兩制』在港的實施，公然挑戰中央政府和香港特區政府的權威。」
林鄭月娥提到，另一方面，部分政黨人士多次揚言要癱瘓香港特區政府，更有部分人士乞求外國政府干預香港事務，甚至對香港實施制裁，這些行為已嚴重觸碰「一國」原則的底線，破壞中央人民政府和特區的關係、危害中國的主權和國家安全以及挑戰中央政府和《基本法》的權威，「我們不可能以為中央政府會對這些行為視而不見。全國人大制定特區維護國家安全法律的《決定》絕非一件輕率的事，而是對香港情況果斷而直接的回應，也是為恢復香港穩定和維護人民利益而急需作出。」
林鄭月娥指出，全國人大作出如此重要的《決定》，大家自然希望了解《決定》對香港未來的影響，對許多在香港的跨國企業來說，最關注的是香港日後是否繼續是跨國企業發展的理想地方，「對此，我可以肯定的回答 『是』。」
林鄭月娥提到，香港不會因為單方面的制裁威脅而過份憂慮，香港將繼續依靠自身的基本優勢，包括法治、司法獨立、自由開放的貿易政策、公平競爭環境、資本自由流通等，亦繼續會是受歡迎、堅韌和具競爭力的經濟體，在聯繫國際和國內市場與企業中擔當不可或缺的角色。
林鄭月娥表示，她不會否認，自去年六月起不斷升級的暴力和別人認為香港未必安全的觀感，已損害了香港的競爭力和國際聲譽，由今年年初開始，香港在不同方面的國際排名有所下降，其中美國的傳統基金會發布的最新經濟自由指數，香港獲評為全球第二，「香港過去四分一世紀都在指數中居首，失去首位固然令人失望，但並不意外，而得分下跌的原因主要是由於治安問題，拖低香港在投資自由方面的分數。」
林鄭月娥指出，在最新發布的《全球金融中心指數》，香港的排名由第三跌至第六，但香港在金融服務方面的優勢仍然明顯，包括貨幣自由兌換

▲全聯將以1千家門市為據點，推出實體電商服務，10月還有機車宅配服務。（圖／資料照） 
記者林育綾／台北報導
隨著「振興三倍券」即將上路，量販業者全聯也將加碼推出「振興大禮包」，除了使用PX Pay會員以實體振興券單筆購物滿1000元，就送福利點1000點（價值100元），加贈「價值3萬元」的振興大禮包，內含食衣住行育樂超過50個品牌優惠。全聯同時也宣布將推出實體電商服務，7月上線試營運，10月還有機車宅配服務。
全聯在先前疫情期間，帶動一波在家煮風潮，生鮮類業績成長30％，冷凍食品、調理熟食、便利醬料包等商品業績也同步成長，而他們表示，身為台灣本土企業一直以來堅持低價、微利經營策略，商品定價比市場行情便宜20%，並強調物超所值，在消費者心中建立「全民超市」的角色，門市更從早期的66家，至2020年已累積超過1000家，去年業績更創下1300億元新高紀錄。
同時在疫情期間，為了推廣消費大眾使用「PX Pay」APP非接觸性支付工具，即日起至年底推出點數2倍送活動，使用PX Pay消費滿100元送6福利點（含原3點），消費者亦可將實體福利卡點數轉移至PX Pay合併，更開通使用PX Pay 即可以5秒時間，快速登入「PX Go！」，此舉更吸引PX Pay下載數即將突破700萬！
如今為了響應政府「振興三倍券」政策，全聯也宣布自7/15起至8/13推出加碼優惠，PX Pay會員持實體振興券單筆購物滿1000元，就送福利點1000點（價值100元）；而持實體卡的會員，則是使用實體振興券全額消費滿1000元送500點；兩者同時再限量加贈超過3萬元價值的「振興大禮包」。
「振興大禮包」內含食衣住行育樂超過50個品牌優惠，尤其為了共同振興國內旅遊，還特別洽談了國立傳藝藝術中心、六福村、九族文化村等育樂優惠，大禮包限量100萬份，送完為止。
▲全聯「振興大禮包」內含食衣住行育樂超過50個品牌優惠，特別洽談了國立傳藝藝術中心、六福村、九族文化村等。（圖／國立傳藝藝術中心，全聯提供）
全聯同時也將整合線上線下服務，以超過1000家門市距離優勢為基礎，推出實體電商服務，以各門市為核心並涵蓋附近範圍，讓消費者可透過線上購物、付款，再到實體店面取貨，屆時生鮮品項最快只要2個小時就能取得。例如忙碌的上班族，可以在下班前用PX GO！先訂貨，下班再順路到家裡附近的全聯取貨，不僅方便還能確保商品